In [1]:
import os

In [2]:
%pwd

'f:\\artificial intelegnce\\study\\ML End To End Projects Krish Naik\\github\\End-To-End_Rice-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\artificial intelegnce\\study\\ML End To End Projects Krish Naik\\github\\End-To-End_Rice-Classification-Project'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ProcessedDataConfig:
    root_dir: Path
    data_dir:Path
    train_dir: Path
    processed_train_data_pickle_file: Path
    valid_dir: Path
    processed_valid_data_pickle_file: Path
    test_dir: Path
    processed_test_data_pickle_file: Path
    params_image_fixed_size: tuple
    

In [6]:
from RiceClassifier.constants import *
from RiceClassifier.utils.common import read_yaml, create_directories
import os

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_processed_data_config(self) -> ProcessedDataConfig:
        config = self.config.processed_data
        data_dir = os.path.join(self.config.data_ingestion.unzip_dir,'Rice_Images')
        create_directories([config.root_dir])
        
        processed_data_config = ProcessedDataConfig(
            root_dir=Path(config.root_dir),
            data_dir=Path(data_dir),
            train_dir=Path(config.train_dir),
            processed_train_data_pickle_file=Path(config.processed_train_data_pickle_file),
            valid_dir=Path(config.valid_dir),
            processed_valid_data_pickle_file=Path(config.processed_valid_data_pickle_file),
            test_dir=Path(config.test_dir),
            processed_test_data_pickle_file=Path(config.processed_test_data_pickle_file),
            params_image_fixed_size=self.params.IMAGE_FIXED_SIZE
            
        )
        
        return processed_data_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import pickle
from ensure import ensure_annotations
from pathlib import Path
from RiceClassifier.logger import logger
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from RiceClassifier.utils.common import save_bin


In [9]:
""" class ProcessedData:
    def __init__(self, config: ProcessedDataConfig):
        self.config = config
    
    def get_process_data(self):
        df_images, df_labels = self.read_images()
        X,y = self.resize_image(df_images,df_labels)
        X,y = self.scale_date(X,y)
        X_train, y_train, X_val, y_val, X_test, y_test = self.separate_data(X,y)       
        
        train_dataset_list = [X_train, y_train]
        valid_dataset_list = [X_val, y_val]
        test_dataset_list = [X_test, y_test]
        
        
        print(self.config.processed_train_data_pickle_file)
        print(X_train)
        
        create_directories([Path(self.config.train_dir)]) 
        create_directories([Path(self.config.valid_dir)]) 
        create_directories([Path(self.config.test_dir)]) 
        
        
            
            
        self.save_pickle(train_dataset_list,self.config.processed_train_data_pickle_file)
        self.save_pickle(valid_dataset_list,self.config.processed_valid_data_pickle_file)
        self.save_pickle(test_dataset_list,self.config.processed_test_data_pickle_file)
    
    
    
    def read_images(self):
        data_dir = self.config.data_dir # Datasets path
        data_dir = Path(data_dir)
        
        arborio = list(data_dir.glob('Arborio/*'))[:100]
        basmati = list(data_dir.glob('Basmati/*'))[:100]
        ipsala = list(data_dir.glob('Ipsala/*'))[:100]
        jasmine = list(data_dir.glob('Jasmine/*'))[:100]
        karacadag = list(data_dir.glob('Karacadag/*'))[:100]
                # Contains the images path
        df_images = {
            'arborio' : arborio,
            'basmati' : basmati,
            'ipsala' : ipsala,
            'jasmine' : jasmine,
            'karacadag': karacadag
        }

        # Contains numerical labels for the categories
        df_labels = {
            'arborio' : 0,
            'basmati' : 1,
            'ipsala' : 2,
            'jasmine' : 3,
            'karacadag': 4
        }
        return df_images, df_labels
    
    def resize_image(self,df_images,df_labels):
        X, y = [], [] # X = images, y = labels
        for label, images in df_images.items():
            for image in images:
                img = cv2.imread(str(image))
                resized_img = cv2.resize(img, (224,224)) # Resizing the images to be able to pass on MobileNetv2 model
                X.append(resized_img) 
                y.append(df_labels[label])
        return X,y
    
    def scale_date(self,X1, y1):
        X = np.array(X1)
        X = X/255
        y = np.array(y1)
        return X,y
    
    def separate_data(self,X, y):
        X_train, X_test_val, y_train, y_test_val = train_test_split(X, y)
        X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val)
        
        return X_train, y_train, X_val, y_val, X_test, y_test

    
    
    def save_pickle(self, pickle_file, pickle_path):
        pickle_path =Path(pickle_path)
        print(pickle_path)
        pickle.dump(pickle_file,open(pickle_path,'wb'))
        logger.info(f'Pickle file saved at: {pickle_path}')
    

    def load_pickle(self,pickle_path):
        pickle_path = Path(pickle_path)
        pickle_file = pickle.load(open(pickle_path,'rb'))
        logger.info(f'Pickle file loaded from:{pickle_path}')
        return pickle_file
    
    def get_train_data(self):
        return self.load_pickle(self.config.processed_train_data_pickle_file)
    
    def get_valid_data(self):
        return self.load_pickle(self.config.processed_valid_data_pickle_file)
    
    def get_test_data(self):
        return self.load_pickle(self.config.processed_test_data_pickle_file) """

" class ProcessedData:\n    def __init__(self, config: ProcessedDataConfig):\n        self.config = config\n    \n    def get_process_data(self):\n        df_images, df_labels = self.read_images()\n        X,y = self.resize_image(df_images,df_labels)\n        X,y = self.scale_date(X,y)\n        X_train, y_train, X_val, y_val, X_test, y_test = self.separate_data(X,y)       \n        \n        train_dataset_list = [X_train, y_train]\n        valid_dataset_list = [X_val, y_val]\n        test_dataset_list = [X_test, y_test]\n        \n        \n        print(self.config.processed_train_data_pickle_file)\n        print(X_train)\n        \n        create_directories([Path(self.config.train_dir)]) \n        create_directories([Path(self.config.valid_dir)]) \n        create_directories([Path(self.config.test_dir)]) \n        \n        \n            \n            \n        self.save_pickle(train_dataset_list,self.config.processed_train_data_pickle_file)\n        self.save_pickle(valid_dataset_l

In [10]:

class ProcessedData:
    def __init__(self, config: ProcessedDataConfig):
        self.config = config
    
    def get_process_data(self):
        df_images, df_labels = self.read_images()
        X, y = self.resize_images(df_images, df_labels)
        X, y = self.scale_data(X, y)
        train_data, valid_data, test_data = self.split_data(X, y)

        self.create_directories()
        self.save_pickle(train_data, self.config.processed_train_data_pickle_file)
        self.save_pickle(valid_data, self.config.processed_valid_data_pickle_file)
        self.save_pickle(test_data, self.config.processed_test_data_pickle_file)
    
    def read_images(self):
        data_dir = Path(self.config.data_dir)
        image_paths = {
            'arborio': list(data_dir.glob('Arborio/*'))[:100],
            'basmati': list(data_dir.glob('Basmati/*'))[:100],
            'ipsala': list(data_dir.glob('Ipsala/*'))[:100],
            'jasmine': list(data_dir.glob('Jasmine/*'))[:100],
            'karacadag': list(data_dir.glob('Karacadag/*'))[:100]
        }
        label_mapping = {
            'arborio': 0,
            'basmati': 1,
            'ipsala': 2,
            'jasmine': 3,
            'karacadag': 4
        }
        return image_paths, label_mapping
    
    def resize_images(self, image_paths, label_mapping):
        X, y = [], []
        for label, paths in image_paths.items():
            for path in paths:
                img = cv2.imread(str(path))
                resized_img = cv2.resize(img, (224, 224))
                X.append(resized_img)
                y.append(label_mapping[label])
        return np.array(X), np.array(y)
    
    def scale_data(self, X, y):
        X_scaled = X / 255
        return X_scaled, y
    
    def split_data(self, X, y):
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
        return (X_train, y_train), (X_val, y_val), (X_test, y_test)
    
    def create_directories(self):
        create_directories([Path(self.config.train_dir)]) 
        create_directories([Path(self.config.valid_dir)]) 
        create_directories([Path(self.config.test_dir)]) 
    
    def save_pickle(self, data, pickle_path):
        with open(pickle_path, "wb") as file:
            pickle.dump(data, file)
            logger.info(f'Pickle file saved at: {pickle_path}')
    
    def load_pickle(self, pickle_path):
        with open(pickle_path, "rb") as file:
            data = pickle.load(file)
            logger.info(f'Pickle file loaded from: {pickle_path}')
        return data
    
    def get_train_data(self):
        return self.load_pickle(self.config.processed_train_data_pickle_file)
    
    def get_valid_data(self):
        return self.load_pickle(self.config.processed_valid_data_pickle_file)
    
    def get_test_data(self):
        return self.load_pickle(self.config.processed_test_data_pickle_file)

In [11]:
try:
    config= ConfigurationManager()
    processed_data_config = config.get_processed_data_config()
    processed_data = ProcessedData(config=processed_data_config)
    processed_data.get_process_data()
except Exception as e: 
    raise e

[2023-07-07 21:50:58,968: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-07 21:50:58,972: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-07 21:50:58,974: INFO: common: created directory at: artifacts]
[2023-07-07 21:50:58,975: INFO: common: created directory at: artifacts/processed_data]
[2023-07-07 21:51:01,933: INFO: common: created directory at: artifacts\processed_data\train]
[2023-07-07 21:51:01,934: INFO: common: created directory at: artifacts\processed_data\valid]
[2023-07-07 21:51:01,963: INFO: common: created directory at: artifacts\processed_data\test]
[2023-07-07 21:51:06,433: INFO: 2432684885: Pickle file saved at: artifacts\processed_data\train\processed_train_data.pkl]
[2023-07-07 21:51:08,331: INFO: 2432684885: Pickle file saved at: artifacts\processed_data\valid\processed_valid_data.pkl]
[2023-07-07 21:51:09,392: INFO: 2432684885: Pickle file saved at: artifacts\processed_data\test\processed_test_data.pkl]


In [32]:
[X_train,y_train] =processed_data.get_train_data()

[2023-07-05 21:10:18,946: INFO: 2432684885: Pickle file loaded from: artifacts\processed_data\train\processed_train_data.pkl]


In [33]:
y_train

array([4, 0, 2, 1, 1, 1, 2, 0, 4, 3, 2, 1, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2,
       0, 1, 1, 3, 2, 4, 2, 0, 1, 0, 4, 0, 3, 3, 3, 1, 1, 1, 3, 1, 4, 0,
       3, 1, 0, 0, 3, 1, 4, 2, 4, 1, 2, 4, 1, 3, 1, 4, 2, 1, 3, 0, 1, 2,
       4, 0, 4, 1, 1, 2, 2, 3, 2, 3, 3, 3, 2, 1, 3, 2, 4, 2, 1, 4, 2, 4,
       3, 2, 3, 2, 3, 3, 1, 1, 3, 4, 4, 4, 4, 2, 3, 1, 1, 1, 2, 2, 1, 1,
       3, 2, 4, 3, 1, 2, 4, 1, 3, 3, 1, 4, 0, 4, 0, 2, 0, 2, 1, 3, 1, 0,
       0, 0, 1, 1, 3, 3, 2, 0, 0, 2, 4, 3, 1, 0, 4, 4, 2, 2, 2, 4, 0, 2,
       4, 1, 2, 4, 2, 1, 0, 2, 2, 0, 0, 0, 2, 3, 2, 4, 0, 4, 2, 2, 2, 1,
       4, 1, 0, 1, 1, 4, 0, 3, 4, 0, 3, 0, 1, 0, 3, 4, 4, 2, 2, 4, 4, 3,
       2, 2, 2, 2, 4, 2, 4, 4, 3, 3, 2, 2, 1, 0, 2, 4, 1, 3, 2, 1, 0, 3,
       0, 4, 4, 0, 4, 0, 2, 0, 2, 4, 3, 0, 3, 4, 0, 3, 2, 2, 0, 3, 0, 3,
       2, 1, 3, 4, 1, 3, 4, 3, 2, 0, 3, 0, 4, 1, 4, 1, 4, 1, 1, 0, 0, 3,
       2, 2, 0, 3, 4, 1, 2, 4, 1, 3, 2, 4, 3, 2, 3, 1, 1, 1, 3, 0, 3, 0,
       4, 3, 2, 4, 1, 4, 0, 1, 0, 1, 2, 3, 4, 1])

In [26]:
[X_val,y_val] =processed_data.get_valid_data()

[2023-07-05 21:02:34,183: INFO: 1711934621: Pickle file loaded from:artifacts\processed_data\valid\processed_valid_data.pkl]


In [27]:
y_val

array([0, 2, 2, 2, 2, 2, 3, 1, 2, 4, 1, 2, 1, 4, 0, 1, 0, 1, 3, 1, 4, 1,
       1, 1, 0, 2, 1, 1, 0, 2, 3, 1])

In [34]:
[X_test,y_test] =processed_data.get_test_data()

[2023-07-05 21:17:25,501: INFO: 2432684885: Pickle file loaded from: artifacts\processed_data\test\processed_test_data.pkl]


In [36]:
y_test

array([3, 3, 4, 4, 1, 2, 3, 0, 1, 0, 4, 4, 4, 4, 3, 1, 1, 0, 4, 2, 2, 4,
       2, 3, 0, 4, 3, 0, 2, 4, 4, 3, 0, 3, 0, 3, 0, 0, 4, 2, 4, 0, 2, 0,
       3, 2, 3, 4, 0, 1, 4, 1, 3, 4, 2, 2, 0, 4, 4, 0, 1, 0, 3, 4, 3, 0,
       4, 1, 3, 1, 1, 2, 4, 0, 1, 0, 4, 2, 2, 4, 3, 0, 0, 0, 3, 0, 3, 4,
       4, 2, 2, 2, 4, 1, 1, 1, 0, 3, 2, 2])